## 1 - Import

In [2]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
import urllib.parse
import fitz
import re
import numpy as np

In [3]:
load_dotenv()

True

## 2 - Models

In [4]:
llm_4omini = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint_4mini'), 
    openai_api_version="2024-10-21",   
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key_4mini'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt4o-mini")
    #streaming=True

In [5]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv('Azure_OpenAI_emb_3_large_azure_endpoint'),
    openai_api_key=os.getenv('Azure_OpenAI_emb_3_large_api_key'),
    openai_api_version="2023-05-15",
    chunk_size=1000
)


/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_41849/426928708.py:1: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(


## 3 - Chunking

In [6]:
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

In [7]:
def chunk_stat(chunk):
    chunk_lengths = [len(chunk) for chunk in chunks_aml5]
    print("Longueurs :", chunk_lengths)
    print("Longueur moyenne :", np.mean(chunk_lengths))
    print("Longueur max :", np.max(chunk_lengths))
    print("Longueur min :", np.min(chunk_lengths))

### 3.1 - Input url

In [8]:
url_aml_5="http://publications.europa.eu/resource/celex/32015L0849"
url_crr="http://publications.europa.eu/resource/celex/32013R0575"

In [9]:
url_aml_5_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32015L0849"
url_crr_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32013R0575"

### 3.2 - Url to local pdf

In [10]:
def pipe_1_url_to_pdf(url, save_path):
    response = requests.get(url, 
                            headers={"Accept": "application/pdf"})
    # tres important pour ne pas renvoyer rdf ou autre format...
    with open(save_path, "wb") as f:
        f.write(response.content)

In [11]:
pipe_1_url_to_pdf(url_aml_5_pdf,
                  "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/aml5.pdf")

In [12]:
pipe_1_url_to_pdf(url_crr_pdf,
                     "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/crr.pdf")

### 3.3 - PyMuPDF - Fitz

PyMuPDF is a high-performance Python library for data extraction, analysis, conversion & manipulation of PDF (and other) documents

In [13]:
def pipe_2_extract_text_with_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip() 

In [14]:
url_aml_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/aml5.pdf")

In [15]:
url_crr_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/crr.pdf")

In [16]:
count_tokens(url_aml_pdf_txt_fitz)

45232

In [17]:
count_tokens(url_crr_pdf_txt_fitz)

407103

### 3.4 - data cleaning

In [18]:
def pipe_3_nettoyer_texte(texte):
    # 1. Supprime les caractères de coupure de mot suivis de retour à la ligne
    texte = re.sub(r'\xad\n', '', texte)

    # 2. \ndénommé -> dénommé, \nEn -> En
    texte = re.sub(r'\n(?=\w)', '', texte)

    # 3. Supprime les retours à la ligne devant une parenthèse contenant des chiffres (\n(2) -> (2))
    texte = re.sub(r'\n(?=\(\d+\))', '', texte)

    # 4. Remplace les apostrophes échappées (l\'Union -> l'Union)
    texte = re.sub(r"\\'", "'", texte)

    # 5. Supprime tout autre backslash isolé non nécessaire
    texte = re.sub(r'\\', '', texte)

    return texte

In [19]:
url_crr_pdf_txt_fitz_clean= pipe_3_nettoyer_texte(url_crr_pdf_txt_fitz)
url_aml_pdf_txt_fitz_clean= pipe_3_nettoyer_texte(url_aml_pdf_txt_fitz)

In [20]:
count_tokens(url_aml_pdf_txt_fitz_clean)

42527

In [21]:
count_tokens(url_crr_pdf_txt_fitz_clean)

360213

## 3 - Chunking

### 3.1  - By Article

In [22]:
article_splitter = RecursiveCharacterTextSplitter(
    separators=   ["Article"],
    chunk_size= 100, 
# attention spliter	Si le « morceau » résultant dépasse chunk_size, 
# il redescend au séparateur suivant (plus petit).
    chunk_overlap=0)

In [23]:
chunks_aml5 = article_splitter.split_text(url_aml_pdf_txt_fitz_clean)
chunks_aml5 = chunks_aml5[:70]
chunk_stat(chunks_aml5)
chunks_aml5

Longueurs : [60562, 2328, 6270, 14858, 671, 293, 2868, 2169, 2321, 2132, 645, 1500, 1782, 4407, 2715, 667, 465, 706, 2461, 1402, 1008, 877, 564, 233, 503, 443, 1059, 645, 1034, 308, 4554, 3421, 3965, 1115, 1399, 827, 705, 674, 421, 2828, 2967, 2453, 605, 287, 2147, 5213, 1796, 921, 4539, 706, 238, 1040, 129, 2163, 458, 1317, 737, 322, 2356, 3119, 5158, 1589, 1199, 1025, 1353, 170, 262, 985, 143, 5595]
Longueur moyenne : 2697.1
Longueur max : 60562
Longueur min : 129


["DIRECTIVES DIRECTIVE (UE) 2015/849 DU PARLEMENT EUROPÉEN ET DU CONSEIL du 20 mai 2015 relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme, modifiant le règlement (UE) no 648/2012 du Parlement européen et du Conseil et abrogeant la directive 2005/60/CE du Parlement européen et du Conseil et la directive 2006/70/CE de la Commission \n(Texte présentant de l'intérêt pour l'EEE) LE PARLEMENT EUROPÉEN ET LE CONSEIL DE L'UNION EUROPÉENNE, vu le traité sur le fonctionnement de l'Union européenne, et notamment son article 114, vu la proposition de la Commission européenne, après transmission du projet d'acte législatif aux parlements nationaux, vu l'avis de la Banque centrale européenne (1), vu l'avis du Comité économique et social européen (2), statuant conformément à la procédure législative ordinaire (3), considérant ce qui suit: (1)  Les flux d'argent illicite peuvent nuire à l'intégrité, à la stabilité et à la

In [24]:
chunks_crr = article_splitter.split_text(url_crr_pdf_txt_fitz_clean)
chunks_crr = chunks_crr[1:522]
chunk_stat(chunks_crr)
chunks_crr

Longueurs : [60562, 2328, 6270, 14858, 671, 293, 2868, 2169, 2321, 2132, 645, 1500, 1782, 4407, 2715, 667, 465, 706, 2461, 1402, 1008, 877, 564, 233, 503, 443, 1059, 645, 1034, 308, 4554, 3421, 3965, 1115, 1399, 827, 705, 674, 421, 2828, 2967, 2453, 605, 287, 2147, 5213, 1796, 921, 4539, 706, 238, 1040, 129, 2163, 458, 1317, 737, 322, 2356, 3119, 5158, 1589, 1199, 1025, 1353, 170, 262, 985, 143, 5595]
Longueur moyenne : 2697.1
Longueur max : 60562
Longueur min : 129


["Article premier Champ d'application Le présent règlement fixe des règles uniformes concernant les exigences prudentielles générales que tous les établissements faisant l'objet d'une surveillance en vertu de la directive 2013/36/UE respectent en ce qui concerne: a) les exigences de fonds propres relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de crédit, de risque de marché, de risque opérationnel et de risque de règlement; b) les exigences limitant les grands risques; c) après l'entrée en vigueur de l'acte délégué visé à l'article 460, les exigences de liquidité relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de liquidité; d) les obligations de déclaration en ce qui concerne les points a), b) et c) et le levier; e) les obligations de publication. Le présent règlement ne régit pas les exigences de publication applicables aux autorités compétentes dans le domaine de la régulation et de la surveillance prudentielles 

### 3.2 - By "CHAPITRE", "SECTION", "Article"

In [25]:
article_splitter_v2 = RecursiveCharacterTextSplitter(
    separators=   ["CHAPITRE", "SECTION", "Article"],
    chunk_size= 100, 
# attention spliter	Si le « morceau » résultant dépasse chunk_size, 
# il redescend au séparateur suivant (plus petit).
    chunk_overlap=0)

In [26]:
chunks_v2=article_splitter_v2.split_text(url_aml_pdf_txt_fitz_clean)
chunks_v2= chunks_v2[1:90]
chunk_stat(chunks_v2)
chunks_v2

Longueurs : [60562, 2328, 6270, 14858, 671, 293, 2868, 2169, 2321, 2132, 645, 1500, 1782, 4407, 2715, 667, 465, 706, 2461, 1402, 1008, 877, 564, 233, 503, 443, 1059, 645, 1034, 308, 4554, 3421, 3965, 1115, 1399, 827, 705, 674, 421, 2828, 2967, 2453, 605, 287, 2147, 5213, 1796, 921, 4539, 706, 238, 1040, 129, 2163, 458, 1317, 737, 322, 2356, 3119, 5158, 1589, 1199, 1025, 1353, 170, 262, 985, 143, 5595]
Longueur moyenne : 2697.1
Longueur max : 60562
Longueur min : 129


['CHAPITRE I DISPOSITIONS GÉNÉRALES',
 "SECTION 1 Objet, champ d'application et définitions",
 "Article premier 1.La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme. 2.Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits. 3.Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement: a)  la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis; b)  le fait de dissimuler ou de déguiser la nature, l'origine, l'emplacement, la disposition, le mouveme

### 3.3 - By "CHAPITRE", "SECTION", "Article"

In [27]:
separators_v2 = [
    r"\d+\.\s?[A-Z][a-z]*",  # Correspond aux numéros de paragraphes comme "1.Le ", "2.La ", "3.par" etc.
    "CHAPITRE",
    "SECTION",
    "Article"
]

In [28]:
article_splitter_v3 = RecursiveCharacterTextSplitter(
    separators=  separators_v2,
    chunk_size= 20, 
    is_separator_regex=True,
    chunk_overlap=0)

In [40]:
chunks_v3 = article_splitter_v3.split_text(url_aml_pdf_txt_fitz_clean)
chunks_v3= chunks_v3[2:280]


In [41]:
chunk_stat(chunks_v3)
chunks_v3

Longueurs : [60562, 2328, 6270, 14858, 671, 293, 2868, 2169, 2321, 2132, 645, 1500, 1782, 4407, 2715, 667, 465, 706, 2461, 1402, 1008, 877, 564, 233, 503, 443, 1059, 645, 1034, 308, 4554, 3421, 3965, 1115, 1399, 827, 705, 674, 421, 2828, 2967, 2453, 605, 287, 2147, 5213, 1796, 921, 4539, 706, 238, 1040, 129, 2163, 458, 1317, 737, 322, 2356, 3119, 5158, 1589, 1199, 1025, 1353, 170, 262, 985, 143, 5595]
Longueur moyenne : 2697.1
Longueur max : 60562
Longueur min : 129


['CHAPITRE I DISPOSITIONS GÉNÉRALES ',
 "SECTION 1 Objet, champ d'application et définitions ",
 'Article premier',
 "1.La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme. ",
 '2.Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits. ',
 "3.Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement: a)  la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis; b)  le fait de dissimuler ou de déguiser la nature, l'origine, l'emplacement, la disposi

In [37]:
from langchain.schema import Document

def create_documents_with_metadata(text):
    sections = article_splitter_v3.split_text(text)
    documents = []
    current_metadata = {"chapitre": None, "section": None, "article": None}

    for section in sections:
        lines = section.split('\n')
        for line in lines:
            if line.startswith("CHAPITRE"):
                current_metadata["chapitre"] = line.strip()
            elif line.startswith("SECTION"):
                current_metadata["section"] = line.strip()
            elif line.startswith("Article"):
                current_metadata["article"] = line.strip()
            else:
                # Créer un objet Document avec le contenu de la section et les métadonnées actuelles
                doc = Document(page_content=line.strip(), metadata=current_metadata.copy())
                documents.append(doc)

    return documents

In [42]:
doc = create_documents_with_metadata(chunks_v3)

TypeError: expected string or bytes-like object, got 'list'